In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# data_dir = "/content/drive/My Drive/Deep_Learning_Project/Data"


In [ ]:
# import pandas as pd

# train_data = pd.read_csv(data_dir + "/emnist-letters-train.csv")
# test_data  = pd.read_csv(data_dir + "/emnist-letters-test.csv")

In [ ]:
import os

for root, dirs, files in os.walk('/kaggle/input'):
    for f in files:
        print(os.path.join(root, f))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
test_data = pd.read_csv(r"/kaggle/input/emnist-letters/emnist-letters-test.csv", header=None)
train_data = pd.read_csv(r"/kaggle/input/emnist-letters/emnist-letters-train.csv", header=None)

In [ ]:
column_names = ['label'] + [f'pixel_{i}' for i in range(784)]
train_data.columns = column_names
test_data.columns=column_names

In [ ]:
train_data.head()

In [ ]:
#unique values in label columns
number_of_labels = train_data['label'].nunique()

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
test_data.head()

In [ ]:
def show_samples(X, y, classes=26, n=8):
    plt.figure(figsize=(12,3))
    for i in range(n):
        ax = plt.subplot(1, n, i+1)
        plt.imshow(X[i].squeeze(), cmap='gray')
        lbl = np.argmax(y[i]) if y is not None else None
        if lbl is not None:
            plt.title(chr(ord('A') + lbl))
        plt.axis('off')
    plt.show()

In [ ]:
# Separate features and labels of training
X_train = train_data.drop('label', axis=1).values
y_train_raw = train_data['label'].values


print("Features shape:", X_train.shape)
print("Labels shape:", y_train_raw.shape)


In [ ]:
# Separate features and labels of testing
X_test = test_data.drop('label', axis=1).values
y_test_raw = test_data['label'].values

print("Features shape:", X_test.shape)
print("Labels shape:", y_test_raw.shape)

In [ ]:
# X_train = X_train.to_numpy()
# X_test = X_test.to_numpy()
# Y_train = Y_train.to_numpy()
# Y_test = Y_test.to_numpy()

# Reshape
# X_train = X_train.reshape(-1,28,28,1)
# X_test = X_test.reshape(-1,28,28,1)
# print("x_train shape: ",X_train.shape)
# print("x_test shape: ",X_test.shape)

In [ ]:

X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)
print("x_train shape: ",X_train.shape)
print("x_test shape: ",X_test.shape)


In [ ]:
# X_train = np.transpose(X_train, (0, 2, 1, 3))
# X_train = np.flip(X_train, axis=2)

# X_test = np.transpose(X_test, (0, 2, 1, 3))
# X_test = np.flip(X_test, axis=2)

In [ ]:
show_samples(X_train, y_train_raw, n=5)

In [ ]:
X_train = np.rot90(X_train, k=1, axes=(1,2))
X_test  = np.rot90(X_test,  k=1, axes=(1,2))

In [ ]:
# Normlization
X_train = X_train.astype('float32') / 255.0
X_test  = X_test.astype('float32') / 255.0

In [ ]:
# import cv2
# import numpy as np

# X_train_uint8 = (X_train).astype('uint8')
# X_test_uint8 = (X_test).astype('uint8')

# X_train = np.array([cv2.resize(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB), (32, 32)) for img in X_train_uint8])
# X_test = np.array([cv2.resize(cv2.cvtColor(img, cv2.COLOR_GRAY2RGB), (32, 32)) for img in X_test_uint8])

# print(X_train.shape, X_test.shape)

# One Hot Encoding

In [ ]:
y_train = to_categorical(y_train_raw - 1, num_classes=26)
y_test  = to_categorical(y_test_raw - 1, num_classes=26)

# Shuffling the training dataset

In [ ]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Train & validation split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Training set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)

# Visualizing sum samples

In [ ]:
show_samples(X_train, y_train, n=30)

# Initialize the model

In [ ]:
model_vgg = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(26, activation='softmax')
])


# VGG 19

In [ ]:
# # import tensorflow as tf
# # from tensorflow.keras.models import Sequential
# # from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# def build_vgg19_from_scratch(input_shape=(32, 32, 3), num_classes=26):
#     model_vgg = Sequential()

#     # Block 1
#     model_vgg.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
#     model_vgg.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

#     # Block 2
#     model_vgg.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

#     # Block 3
#     model_vgg.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

#     # Block 4
#     model_vgg.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

#     # Block 5
#     model_vgg.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
#     model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

#     # Fully Connected Layers
#     model_vgg.add(Flatten())
#     model_vgg.add(Dense(4096, activation='relu'))
#     model_vgg.add(Dropout(0.5))
#     model_vgg.add(Dense(4096, activation='relu'))
#     model_vgg.add(Dropout(0.5))
#     model_vgg.add(Dense(num_classes, activation='softmax')) # For classification

#     return model_vgg

# model_vgg = build_vgg19_from_scratch()


# Compile the model

In [ ]:
model_vgg.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=6,
        restore_best_weights=True
    )
]


In [ ]:
history = model_vgg.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)


In [ ]:
# save final model
model.save('final_emnist_vgg.h5')

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Loss')
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.title('Accuracy')
plt.show()

In [ ]:
test_loss, test_acc = model_vgg.evaluate(X_test, y_test, verbose=1)
print("Final Test Loss:", test_loss)
print("Final Test Accuracy:", test_acc)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
y_pred_proba = model_vgg.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, cmap="Blues", annot=True, fmt='d', annot_kws={"size": 10})
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.show()

# Show some misclassified examples

In [ ]:
mis_idx = np.where(y_pred != y_true)[0]
print('Number of misclassified:', len(mis_idx))


n_show = min(12, len(mis_idx))
plt.figure(figsize=(12,6))
for i in range(n_show):
    idx = mis_idx[i]
    ax = plt.subplot(3, 4, i+1)
    plt.imshow(X_test[idx].reshape(28, 28).T, cmap='gray')
    # plt.imshow(X_test[idx].squeeze(), cmap='gray')
    plt.title(f'T:{chr(ord("A")+y_true[idx])} P:{chr(ord("A")+y_pred[idx])}')
    plt.axis('off')
plt.show()

# After optimizing some misclassified examples

In [ ]:
mis_idx = np.where(y_pred != y_true)[0]
np.random.shuffle(mis_idx)

print(f'Total misclassified: {len(mis_idx)}')
n_show = min(12, len(mis_idx))
plt.figure(figsize=(12, 6))
for i in range(n_show):
    idx = mis_idx[i]
    ax = plt.subplot(3, 4, i + 1)
    img = X_test[idx].reshape(28, 28)
    img = np.rot90(img.T, k=1)
    plt.imshow(img, cmap='gray')

    true_char = chr(ord("A") + y_true[idx])
    pred_char = chr(ord("A") + y_pred[idx])

    plt.title(f'True: {true_char} | Pred: {pred_char}', fontsize=10)
    plt.axis('off')

plt.tight_layout()
plt.show()

# This indicates that about 5% of EMNIST letters dataset looks like the messy examples above.

# ResNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.preprocessing import label_binarize

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


In [ ]:
from tensorflow.keras.preprocessing.image import smart_resize

X_train_resnet = smart_resize(X_train, (64, 64))
X_val_resnet = smart_resize(X_val, (64, 64))
X_test_resnet = smart_resize(X_test, (64, 64))


In [ ]:
print("X_train shape:", X_train_resnet.shape)
print("X_val shape:", X_val_resnet.shape)
print("X_test shape:", X_test_resnet.shape)


In [ ]:
num_classes = 26  # EMNIST letters 1–26

# reshape to NHWC and normalize
# X_train = X_train / 255.
# X_val   = X_val / 255.
# X_test  = X_test/ 255.

# y_train_cat = to_categorical(y_train)
# y_val_cat   = to_categorical(y_val)
# y_test_cat  = to_categorical(y_test)

inputs = Input(shape=(64, 64, 1))

# 1) Resize to 224x224 for ResNet
x = layers.Resizing(224, 224)(inputs)

# 2) Convert grayscale → RGB (3 channels)
x = layers.Conv2D(3, (3, 3), padding='same')(x)

In [ ]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False  # basic transfer learning
x = base_model(x, training=False)

# classification head
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

resnet_model = models.Model(inputs, outputs, name="ResNet50_Transfer")


In [ ]:
resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

resnet_model.summary()


In [ ]:
history_resnet = resnet_model.fit(
    X_train_resnet, y_train,
    validation_data=(X_val_resnet, y_val),
    epochs=10,
    batch_size=64,
    verbose=1
)


In [ ]:
y_proba = resnet_model.predict(X_test_resnet)
y_pred = np.argmax(y_proba, axis=1)
y_true = y_test


In [ ]:
# report = classification_report(y_true, y_pred, output_dict=False)
# print(report)


In [ ]:
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=False, cmap="Blues")
plt.title("Confusion Matrix - ResNet50")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
# Binarize labels for ROC
y_test_bin = label_binarize(y_true, classes=np.arange(num_classes))

# Compute AUC
auc_macro = roc_auc_score(y_test_bin, y_proba, average="macro", multi_class="ovr")
auc_micro = roc_auc_score(y_test_bin, y_proba, average="micro", multi_class="ovr")
print("AUC Macro:", auc_macro)
print("AUC Micro:", auc_micro)

# Plot ROC for first few classes
plt.figure(figsize=(10,8))
for i in range(min(num_classes, 5)):  # plot first 5 to avoid clutter
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_proba[:, i])
    plt.plot(fpr, tpr, label=f"Class {i}")

plt.title("ROC Curves (first 5 classes) - ResNet50")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()


In [ ]:
resnet_model.save("resnet50_emnist_transfer.keras")


# MobileNet

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, Input

In [ ]:
# Build MobileNet Model

def build_mobilenet_model(input_shape=(28, 28, 1), num_classes=26):
    inputs = Input(shape=input_shape)

    x = layers.Resizing(32, 32)(inputs)

    x = layers.Conv2D(3, (3, 3), padding='same')(x)

    base_model = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=(32, 32, 3)
    )

    base_model.trainable = True

    x = base_model(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs, name="MobileNet_Custom")
    return model

model = build_mobilenet_model()

In [ ]:
# Compile & Train

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # Low LR for transfer learning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor='val_accuracy'),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2, monitor='val_loss')
]
print(model.summary())

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=64,
    callbacks=callbacks
)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")

In [ ]:
# from sklearn.metrics import classification_report

# report = classification_report(y_true, y_pred, target_names=class_names)
# print("Classification Report:\n")
# print(report)

In [ ]:
unique_labels, counts = np.unique(y_test_raw, return_counts=True)
print("Unique labels in test set:", unique_labels)
print("Counts of each label:", dict(zip(unique_labels, counts)))

Observation:
The test dataset used for evaluation is incomplete. It only contains samples for the first 19 classes (labels 1 to 19, corresponding to letters A through S).

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12, 10))
class_names = [chr(ord('A') + i) for i in range(26)]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
incorrect_indices = np.where(y_pred != y_true)[0]

plt.figure(figsize=(15, 5))
for i, incorrect_idx in enumerate(incorrect_indices[:9]):
    ax = plt.subplot(1, 9, i + 1)
    plt.imshow(X_test[incorrect_idx].squeeze(), cmap='gray')
    true_label = chr(ord('A') + y_true[incorrect_idx])
    pred_label = chr(ord('A') + y_pred[incorrect_idx])
    plt.title(f'True: {true_label}\nPred: {pred_label}', color='red')
    plt.axis('off')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [ ]:
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=callbacks
)

In [ ]:
for layer in model.layers[-40:]:
    if hasattr(layer, "trainable"):
        layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("\nFine-Tuning started...\n")

history_ft = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32,
    callbacks=callbacks
)

In [ ]:
# Data Augmentation

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=7,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.1
)

datagen.fit(X_train)

In [ ]:
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=callbacks
)

## VIT

In [ ]:
# Vision Transformer (ViT) for EMNIST letters (28x28x1 → 26 classes)

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

def create_vit_classifier(
    input_shape=(28, 28, 1),
    num_classes=26,
    image_size=28,
    patch_size=4,           # 28 / 4 = 7 → 49 patches
    projection_dim=64,
    transformer_layers=4,
    num_heads=4,
    transformer_units=[128, 64],   # MLP inside transformer
    mlp_head_units=[256, 128],     # final classification head
    dropout_rate=0.1
):
    inputs = keras.Input(shape=input_shape)

    # 1) Patch embedding using a Conv2D with stride = patch_size
    # Output shape: (batch, 7, 7, projection_dim) for 28x28 with patch_size=4
    x = layers.Conv2D(
        filters=projection_dim,
        kernel_size=patch_size,
        strides=patch_size,
        padding="valid"
    )(inputs)  # (batch, H', W', projection_dim)

    # Flatten patches: (batch, num_patches, projection_dim)
    num_patches = (image_size // patch_size) ** 2  # 49
    x = layers.Reshape((num_patches, projection_dim))(x)

    # 2) Add learnable positional embeddings
    positions = tf.range(start=0, limit=num_patches, delta=1)
    pos_embedding_layer = layers.Embedding(
        input_dim=num_patches,
        output_dim=projection_dim
    )
    position_embeddings = pos_embedding_layer(positions)  # (num_patches, projection_dim)
    x = x + position_embeddings  # broadcast over batch

    # 3) Transformer encoder blocks
    for _ in range(transformer_layers):
        # Layer norm before attention
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)

        # Multi-head self-attention
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=projection_dim,
            dropout=dropout_rate
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, x])

        # Layer norm + MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        for units in transformer_units:
            x3 = layers.Dense(units, activation="gelu")(x3)
            x3 = layers.Dropout(dropout_rate)(x3)

        # Second skip connection
        x = layers.Add()([x2, x3])

    # 4) Classification head
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    # Global average pooling over the patch tokens
    x = layers.GlobalAveragePooling1D()(x)

    # MLP head
    for units in mlp_head_units:
        x = layers.Dense(units, activation="gelu")(x)
        x = layers.Dropout(dropout_rate)(x)

    # Final classifier
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name="vit_emnist_letters")
    return model

# Instantiate the ViT model
model = create_vit_classifier(
    input_shape=(28, 28, 1),
    num_classes=26
)

# Compile it (you can tweak optimizer/lr later)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode='nearest'
)

datagen.fit(X_train)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    validation_data=(X_val, y_val),
    epochs=20,
    callbacks=callbacks
)


# inception V1

In [424]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub

inputs = keras.Input(shape=(28, 28, 1))

# Resize and convert to RGB
x = layers.Resizing(224, 224)(inputs)
x = layers.Conv2D(3, (3,3), padding="same")(x)

# TF-Hub layer
base = hub.KerasLayer(
    "https://tfhub.dev/google/imagenet/inception_v1/feature_vector/5",
    trainable=False
)

# Feed as a dict to avoid symbolic error
x = base({"images": x})

# Classification head
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model_inception = keras.Model(inputs, outputs)
model_inception.summary()


ValueError: Exception encountered when calling layer 'keras_layer_4' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_4' (type KerasLayer):
  • inputs={'images': '<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_1381>'}
  • training=None

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=4,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=2
    ),
    keras.callbacks.ModelCheckpoint(
        "inception_v1_best.keras",
        monitor="val_accuracy",
        save_best_only=True
    )
]

history_inception = model_inception.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    epochs=20,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)


In [ ]:
test_loss, test_acc = model_inception.evaluate(X_test, y_test, verbose=0)
print(f"Inception-V1 Accuracy: {test_acc:.4f}")

y_proba_inc = model_inception.predict(X_test, batch_size=64)
y_pred_inc = np.argmax(y_proba_inc, axis=1)
y_true_inc = np.argmax(y_test, axis=1)

report_inc = classification_report(
    y_true_inc,
    y_pred_inc,
    target_names=class_names,
    output_dict=True
)
pd.DataFrame(report_inc).transpose()


In [ ]:
cm_inc = confusion_matrix(y_true_inc, y_pred_inc)

plt.figure(figsize=(10,8))
sns.heatmap(
    cm_inc,
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names
)
plt.title("Inception-V1 Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
y_true_bin_inc = label_binarize(y_true_inc, classes=np.arange(num_classes))

auc_per_class_inc = roc_auc_score(
    y_true_bin_inc,
    y_proba_inc,
    multi_class='ovr',
    average=None
)

macro_auc_inc = roc_auc_score(
    y_true_bin_inc,
    y_proba_inc,
    multi_class='ovr',
    average='macro'
)

print("Macro AUC:", macro_auc_inc)


In [425]:
import torch
import torchvision
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)


Torch: 2.6.0+cu124
Torchvision: 0.21.0+cu124


In [426]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models, transforms
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [427]:
class EMNISTDataset(Dataset):
    def __init__(self, df, transform=None):
        self.labels = df.iloc[:, 0].values - 1   # letters are 1–26 → convert to 0–25
        self.images = df.iloc[:, 1:].values.astype(np.uint8)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = self.images[idx].reshape(28, 28)   # grayscale
        img = np.stack([img, img, img], axis=2)  # 3 channels

        img = Image.fromarray(img)

        if self.transform:
            img = self.transform(img)

        label = torch.tensor(self.labels[idx]).long()
        return img, label


In [428]:
train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

test_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])


In [429]:
full_ds = EMNISTDataset(train_data, transform=train_tf)

n_total = len(full_ds)
n_train = int(0.8 * n_total)
n_val   = int(0.1 * n_total)
n_test  = n_total - n_train - n_val

train_ds, val_ds, test_ds = random_split(
    full_ds, [n_train, n_val, n_test],
    generator=torch.Generator().manual_seed(123)
)

val_ds.dataset.transform = test_tf
test_ds.dataset.transform = test_tf

print("Train:", len(train_ds), "Val:", len(val_ds), "Test:", len(test_ds))


Train: 71040 Val: 8880 Test: 8880


In [430]:
batch_size = 64

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False)


In [431]:
model = models.googlenet(
    weights=models.GoogLeNet_Weights.IMAGENET1K_V1,
    aux_logits=True
)

num_classes = 26

# main classifier
model.fc = nn.Linear(model.fc.in_features, num_classes)

# auxiliary classifiers
model.aux1.fc2 = nn.Linear(model.aux1.fc2.in_features, num_classes)
model.aux2.fc2 = nn.Linear(model.aux2.fc2.in_features, num_classes)

model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/googlenet.py:341: UserWarning: auxiliary heads in the pretrained googlenet model are NOT pretrained, so make sure to train them
  warnings.warn(


In [432]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3)

best_acc = 0
best_model_path = "googlenet_emnist_best.pth"
epochs = 10


In [433]:
for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs, aux1, aux2 = model(imgs)

        loss = (
            criterion(outputs, labels) +
            0.3 * criterion(aux1, labels) +
            0.3 * criterion(aux2, labels)
        )

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    # validation
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)

            out = model(imgs)
            preds = out.argmax(dim=1)

            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    scheduler.step(1 - val_acc)

    print(f"Epoch {epoch+1}: Train Acc={train_acc:.3f}, Val Acc={val_acc:.3f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), best_model_path)
        print("Saved best model!")


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 15.12 MiB is free. Process 4143 has 15.87 GiB memory in use. Of the allocated memory 396.05 MiB is allocated by PyTorch, and 13.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model.load_state_dict(torch.load(best_model_path))
model.eval()

test_correct = 0
test_total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)

        out = model(imgs)
        preds = out.argmax(dim=1)

        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

print("Test Accuracy:", test_correct / test_total)
